In [1]:
import os
import random
import shutil
import sys

libri_path = "../../../LibriTTS" # absolute path to TensorFlowTTS.
dataset_path = "../../../libritts" # Change to your paths. This is a output of re-format dataset.
subset = "train-clean-100"

In [2]:
with open(os.path.join(libri_path, "SPEAKERS.txt")) as f:
    data = f.readlines()
    
dataset_info = {}
max_speakers = 100  # Max number of speakers to train on
min_len = 10  # Min len of speaker narration time
max_file_len = 15  # max audio file lenght
min_file_len = 2  # min audio file lenght

In [3]:
possible_dataset = [i.split("|") for i in data[12:] if i.split("|")[2].strip() == subset and float(i.split("|")[3].strip()) >= min_len]


In [4]:
ids = [i[0].strip() for i in possible_dataset]


In [5]:
import soundfile as sf

In [6]:
possible_map = {}
subset_path = os.path.join(libri_path, subset)
for i in os.listdir(subset_path):
    if i in ids:
        id_path = os.path.join(subset_path, i)
        id_dur = 0
        id_included = []
        
        for k in os.listdir(id_path):
            for j in os.listdir(os.path.join(id_path, k)):
                if ".wav" in j:
                    f_path = os.path.join(id_path, k, j)
                    sf_file =  sf.SoundFile(f_path)
                    dur = len(sf_file) / sf_file.samplerate
                    if max_file_len < dur or dur < min_file_len:
                        continue
                    else:
                        id_included.append(f_path)
                        id_dur += dur
        possible_map[i] = {"dur": id_dur, "included": id_included}


In [7]:
min_len

10

In [8]:
poss_speakers = {k: v["included"] for k, v in possible_map.items() if v["dur"]/60 >= min_len}

In [9]:
poss_speakers.keys()

dict_keys(['4406', '2289', '831', '5022', '7511', '7059', '4195', '3240', '3982', '8630', '196', '4680', '2416', '4397', '7800', '6209', '6385', '2836', '8838', '1088', '5867', '4267', '8088', '3830', '7178', '2843', '4297', '460', '78', '1970', '4137', '3526', '669', '4018', '8123', '6019', '1098', '412', '1502', '83', '1116', '8324', '5778', '7190', '5393', '8468', '4362', '3486', '6078', '1743', '4051', '7517', '5163', '7367', '4160', '7067', '6836', '6437', '40', '3259', '6064', '3664', '4788', '8312', '405', '4214', '1246', '3857', '2136', '2092', '8051', '5750', '1841', '374', '8098', '6272', '6818', '8238', '2196', '6415', '8419', '696', '1263', '6367', '446', '200', '8108', '5322', '887', '6531', '3879', '6147', '7447', '1553', '2952', '3242', '8580', '5561', '6529', '1624', '8770', '5339', '8747', '87', '6476', '6081', '250', '7505', '6454', '2911', '2893', '7794', '7402', '3235', '7078', '1355', '4830', '8465', '7302'])

In [10]:
to_move = list(poss_speakers.keys())
random.shuffle(to_move)
to_move = to_move[:max_speakers]

In [11]:
for sp_id, v in poss_speakers.items():
    if sp_id in to_move:
        for j in v:
            f_name = j.split(os.path.sep)[-1]
            text_f_name = f_name.split(".wav")[0] + ".txt"
            os.makedirs(os.path.join(dataset_path, sp_id), exist_ok=True)
            shutil.copy(j, os.path.join(dataset_path, sp_id, f_name))
            shutil.copy(j.replace(".wav", ".normalized.txt"), os.path.join(dataset_path, sp_id, text_f_name))

In [12]:
sorted(to_move)

['1088',
 '1098',
 '1116',
 '1246',
 '1263',
 '1355',
 '1502',
 '1553',
 '1624',
 '1743',
 '1841',
 '196',
 '1970',
 '200',
 '2136',
 '2196',
 '2289',
 '250',
 '2843',
 '2893',
 '2911',
 '2952',
 '3235',
 '3240',
 '3242',
 '3259',
 '3486',
 '3526',
 '3664',
 '374',
 '3830',
 '3857',
 '3982',
 '40',
 '405',
 '4051',
 '4137',
 '4160',
 '4214',
 '4267',
 '4297',
 '4362',
 '4397',
 '4406',
 '4680',
 '4830',
 '5022',
 '5163',
 '5322',
 '5339',
 '5393',
 '5561',
 '5750',
 '5867',
 '6019',
 '6064',
 '6078',
 '6081',
 '6147',
 '6272',
 '6385',
 '6415',
 '6437',
 '6454',
 '6476',
 '6529',
 '6531',
 '669',
 '6818',
 '6836',
 '7059',
 '7067',
 '7078',
 '7178',
 '7190',
 '7302',
 '7367',
 '7402',
 '7505',
 '7511',
 '78',
 '7800',
 '8051',
 '8088',
 '8098',
 '8108',
 '8123',
 '83',
 '831',
 '8312',
 '8324',
 '8419',
 '8465',
 '8468',
 '8580',
 '8630',
 '87',
 '8747',
 '8770',
 '8838']

In [13]:
!ls ../../../

LICENSE			docker-compose.yml  preprocess	       tensorflow_tts
LibriTTS		dockerfile	    setup.cfg	       test
README.md		libritts	    setup.py	       ttsexamples
TensorFlowTTS.egg-info	notebooks	    tacotron_libritts
